In [ ]:
import torch
import pandas as pd
import numpy as np  
import re

dfEclipse = pd.read_csv("https://raw.githubusercontent.com/Bookinus/Predicting-Severity-in-Bug-Reports/refs/heads/main/DataSet/ThesisDataSet/Eclipse.csv")
dfMozilla = pd.read_csv("https://raw.githubusercontent.com/Bookinus/Predicting-Severity-in-Bug-Reports/refs/heads/main/DataSet/ThesisDataSet/Mozilla.csv")
dfOpenOffice = pd.read_csv("https://raw.githubusercontent.com/Bookinus/Predicting-Severity-in-Bug-Reports/refs/heads/main/DataSet/ThesisDataSet/Openoffice.csv")

dfEclipse.drop("bugID", axis="columns", inplace= True)
dfMozilla.drop("bugID", axis="columns", inplace= True)
dfOpenOffice.drop("bugID", axis="columns", inplace= True)

dfEclipse.drop_duplicates(inplace=True)
dfMozilla.drop_duplicates(inplace=True)
dfOpenOffice.drop_duplicates(inplace=True)

In [14]:
print(dfEclipse)

                                        TextDescription                  cl  \
0               PDE quickfix creates invalid @Since tag     Eclipse Project   
1     Grant access to projects storage service to th...  Eclipse Foundation   
2                  Add relation information to REST-API          Automotive   
3     Provide platform independent plug-in to set th...     Eclipse Project   
4     Inline method refacting reports "Inaccurate re...     Eclipse Project   
...                                                 ...                 ...   
8473                            JIPP for tools.shellwax  Eclipse Foundation   
8474  [13] Highlighting is broken when Unicode equiv...     Eclipse Project   
8475            duplicate configure fetch/push commands          Technology   
8476             Mailing list for Cargo Tracker project  Eclipse Foundation   
8477         Provide convenient MultiStatus constructor     Eclipse Project   

             pd            co   rp          OS     

In [15]:
dfEclipse = dfEclipse[dfEclipse.Severity != 'enhancement']

print(dfEclipse)

                                        TextDescription                  cl  \
0               PDE quickfix creates invalid @Since tag     Eclipse Project   
1     Grant access to projects storage service to th...  Eclipse Foundation   
2                  Add relation information to REST-API          Automotive   
3     Provide platform independent plug-in to set th...     Eclipse Project   
4     Inline method refacting reports "Inaccurate re...     Eclipse Project   
...                                                 ...                 ...   
8473                            JIPP for tools.shellwax  Eclipse Foundation   
8474  [13] Highlighting is broken when Unicode equiv...     Eclipse Project   
8475            duplicate configure fetch/push commands          Technology   
8476             Mailing list for Cargo Tracker project  Eclipse Foundation   
8477         Provide convenient MultiStatus constructor     Eclipse Project   

             pd            co   rp          OS     